Download a single country from [ParlaMint-4.0](https://www.clarin.si/repository/xmlui/handle/11356/1859):

In [ ]:
! curl --remote-name-all https://www.clarin.si/repository/xmlui/bitstream/handle/11356/1859/ParlaMint-BA.tgz

Uncompress the files:

In [ ]:
! tar -xzvf ParlaMint-BA.tgz

Let's open a single file, split it into sentences, and assign sentiment to them:

In [1]:

import pandas as pd

df = pd.read_csv("ParlaMint-BA.txt/1998/ParlaMint-BA_1998-11-26-0.txt", sep="\t", names=["utterance", "text"])

In [ ]:
!pip install classla

In [2]:
import classla

classla.download("hr")
nlp = classla.Pipeline("hr")
doc = nlp(df.text[0])


2024-06-10 10:19:42 INFO: Downloading these customized packages for language: hr (Croatian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2024-06-10 10:19:42 INFO: File exists: /home/peterr/classla_resources/hr/pos/standard.pt.
2024-06-10 10:19:43 INFO: File exists: /home/peterr/classla_resources/hr/lemma/standard.pt.
2024-06-10 10:19:43 INFO: File exists: /home/peterr/classla_resources/hr/depparse/standard.pt.
2024-06-10 10:19:43 INFO: File exists: /home/peterr/classla_resources/hr/ner/standard.pt.
2024-06-10 10:19:44 INFO: File exists: /home/peterr/classla_resources/hr/pretrain/standard.pt.
2024-06-10 10:19:44 INFO: Finished downloading models and saved to /home/peterr/classla_resources.
2024-06-10 10:19:44 INFO: Loading these models for language: hr (Croatian):
| Processor | Package  |
------------------------
| tokenize  | sta

In [5]:
from conllu import parse

parsed = parse(doc.to_conll())

In [8]:
parsed[0].metadata

{'newpar id': '1',
 'sent_id': '1.1',
 'text': 'Dame i gospodo predlažem da počnemo sa radom.'}